* keras `ImageDataGenerator` class works for 2D augmentation. 
* This code performs runtime (on-the-fly) 3D augmentation. 
* It utilizes the `DataGenerator` class described in https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

## How to use the code?

In [1]:
from aug_data_generator import DataGenerator
import scipy.io as sio
import random

Using TensorFlow backend.


#### Load dataset

In [2]:
dataset = sio.loadmat('dataset.mat')

In [3]:
x = dataset['x'] # data

In [4]:
x.shape

(20, 64, 64, 64, 1)

In [5]:
y = dataset['y'] # label

In [6]:
y.shape

(20, 64, 64, 64, 1)

#### Set parameters

To do on-the-fly augmentation, set the `do_augmentation` to true. </br>
Currently, it performs two augmentations - flip and rotation. </br></br>
Example of how to use rotation:</br>
'rotate_angle': random.sample(range(-15, 15), 3) </br>
It will create 3 random rotation angles within the given range. Thus, the size of the rotation angle will be 1x3. For instance, (-10,5,12). Then it will rotate the data about x, y, and z axes by -10, 5, and 12 respectively. </br></br>
 
Example of how to use flip:</br>
To flip a data, the user needs to define the `flip_axis` parameter. It could be 1, 2, 3, or 'random'. For 'random', axis will be determined randomly. 

In [7]:
# Parameters
params = {'dim': (64,64,64),
          'batch_size': 3,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': False,
          'to_category': True, # set it to true if you want categorical format, otherwise false.
          'do_augmentation': True, # to do on-the-fly augmentation, set it to true. 
          'aug_list': ['rotate', 'flip'], # add types of augmenation you want. 
          # 'rotate_axis': 1,
          'rotate_angle': random.sample(range(-30, 30), 3), # define rotation angles
          'flip_axis': 'random', # set 1,2,3, or 'random'
          }

#### Create instances

In [8]:
train_x, train_y = x[0:15], y[0:15] # training data
val_x, val_y = x[15:20], y[15:20] # validation data

training_generator = DataGenerator(train_x, train_y, **params)
validation_generator = DataGenerator(val_x, val_y, **params)

Now, these two can be used in keras `fit_generator`

In [9]:
# Example
history=model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            steps_per_epoch=len(train_x)//batch_size, 
                            verbose=1,
                            epochs=200,
                            )

#### Additional

If you want to check the augmentation manually, you can do it in the following way. 

In [10]:
training_generator

In [11]:
g, p = training_generator[0]

In [12]:
g.shape

(3, 64, 64, 64, 1)

In [13]:
p.shape

(3, 64, 64, 64, 2)

In [14]:
sio.savemat('aug.mat', {'x':x[0:3], 'y':y[0:3], 'g':g, 'p':p})


# Uncomment to check it using iterator 
# a = iter(training_generator),
# x,y = next(a),
